In [ ]:
# Spikeinterface imports (could do this cleaner, oh well)
import spikeinterface as si
import spikeinterface.extractors as se
import spikeinterface.sorters as ss
import spikeinterface.preprocessing as spre
import spikeinterface.exporters as sxp
import spikeinterface.widgets as sw
import probeinterface as pi
import spikeinterface.curation as scur

# Core python imports
import os
import time as pytime
from IPython.display import Audio
import numpy as np
import matplotlib.pyplot as plt

### Read in data, attach probe file to recording

In [ ]:
## If reading series of intan recordings
rec_name = 'my_recording_name'
path_to_folder = os.path.normpath('C:/Users/myLogin42069/Documents/my_experiment/my_trial/' + rec_name)
dircontents = os.listdir(path_to_folder)
file_names = [x for x in dircontents if '.rhd' in x]
# Make sure arranged in correct order
def extract_number(name):
    return int(name.split('_')[-1].split('.')[0])
file_names = sorted(file_names, key=extract_number)
recording_list = []
for file in file_names:
    path_to_file = os.path.join(path_to_folder, file)
    recording_list.append(se.IntanRecordingExtractor(path_to_file, stream_id='0'))
recording = si.concatenate_recordings(recording_list)
display(recording)

## Uncomment to grab just a section of a recording
# recording = recording.frame_slice(start_frame=0, end_frame=int(231*30000))

## If reading open ephys recording session
# path_to_folder = Path('C:/Users/myLogin42069/Documents/my_experiment/my_trial/')
# recording = se.read_openephys(path_to_folder, block_index=0, stream_id='0')
# recording = si.SelectSegmentRecording(recording, 0)

In [ ]:
# Remove analog input channels if present, not needed for spike sorting
if any('ADC' in s for s in recording.get_channel_ids()):  
    recording = recording.remove_channels([x for x in recording.get_channel_ids() if 'ADC' in x])
recording.get_channel_ids()

In [ ]:
probe = pi.read_probeinterface('A32_A1x32-Poly5-6mm-35s-100.json')
pi.plotting.plot_probe_group(probe, with_channel_index=True, with_device_index=True)
recording.set_probe(probe.probes[0], in_place=True)
recording.get_probe()

### Preprocess: filter and z-score data

If using mountainsort5, filter then zscore with mean+std recommended.
Whitening is not a good idea if channels are too physically close and highly correlated, but mountainsort5 expects unit variance so zscoring is essential

In [ ]:
recording_filter = spre.bandpass_filter(recording, freq_min=300, freq_max=5000)
recording_zscore = spre.zscore(recording_filter, mode='mean+std')

## Run Sorter 
(Mountainsort5 in particular)

In [ ]:
ss.available_sorters()

In [ ]:
sorter = 'mountainsort5'
print(ss.get_default_sorter_params(sorter))
ss.get_sorter_params_description(sorter)

In [ ]:
params = ss.get_default_sorter_params(sorter)
params['scheme'] = '2'
# Parameters for any scheme
params['detect_sign'] = 1
params['detect_threshold'] = 4.0
params['snippet_T1'] = 40
params['snippet_T2'] = 40
params['npca_per_channel'] = 4
params['npca_per_subdivision'] = 20
# Parameters for scheme 2
params['scheme2_phase1_detect_channel_radius'] = 150
params['snippet_mask_radius'] = 200
params['scheme2_training_duration_sec'] = 400
params['scheme2_max_num_snippets_per_training_batch'] = 10000
# Parameters for scheme 3
params['scheme3_block_duration_sec'] = 600

tic = pytime.perf_counter()
sort = ss.run_sorter(
    sorter,
    recording=recording_zscore,
    output_folder=sorter,
    verbose=False,
    **params)
print(f'{pytime.perf_counter()-tic} seconds elapsed')
print(sort)
pytime.sleep(1)
Audio('notification-sound.wav', autoplay=True)

### Create sortinganalyzer

In [ ]:
job_kwargs = dict(n_jobs=8, chunk_duration="1s", progress_bar=True)
analyzer = si.create_sorting_analyzer(sorting=sort, recording=recording_zscore, **job_kwargs)
analyzer.compute(['random_spikes', 'waveforms', 'templates', 'unit_locations'], **job_kwargs)

### Save to Phy

In [ ]:
phy_save_path = './phy_folder/' + sorter + '_' + rec_name
sxp.export_to_phy(sorting_analyzer=analyzer, output_folder=phy_save_path, remove_if_exists=True, **job_kwargs)
# Save record of params
with open(phy_save_path+'/params_log.txt', 'w') as f: 
    for key, value in params.items(): 
        f.write('%s:%s\n' % (key, value))

 To run Phy, use cmd or Powershell, and do one of the following:
 
 1. Run the command spit out by the cell above, often something like:
 
 ```phy template-gui  C:\Users\lwood39\Documents\AutoSpikeSort\phy_folder\mountainsort5_poke1_230520_153135\params.py```
 
 
 2. Navigate to phy_folder created by the above cell, then run phy command
 
 Example:
 
 ```cd C:/Users/lwood39/Documents/AutoSpikeSort/phy_folder_kilosort```
 
 ```phy template-gui params.py```
 

# Further plotting or exploration:

In [ ]:
# To plot traces straight:
# sw.plot_timeseries(
#     recording_cache_raw, 
#     time_range=(69, 420), 
#     channel_ids=['0'], 
#     return_scaled=True)
# plt.show()

# To get traces of a specific channel from specific frames:
# data = recording_cache_raw.get_traces(
#     start_frame=your_start_frame_here, 
#     end_frame=your_end_frame_here,
#     return_scaled=True, 
#     channel_ids=['3'])